In [15]:
from dotenv import load_dotenv
import os

# Substitua abaixo com o caminho real do seu .env
caminho_env = "/content/drive/MyDrive/Segredos/.env"
load_dotenv(dotenv_path=caminho_env)

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")
telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

print("client_id:", client_id)
print("client_secret:", client_secret)

client_id: None
client_secret: None


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
from dotenv import load_dotenv
import os

caminho_env = "/content/drive/MyDrive/Segredos/.env"  # Caminho absoluto
load_dotenv(dotenv_path=caminho_env)

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")
telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

print("client_id:", client_id)
print("client_secret:", client_secret)

client_id: QQX5b-hZ
client_secret: 7rOhlA2OaGCnLf_Y7nz6a4buz_jvkuCNC4Hu0pkfOyU


In [18]:
token = conectar_deribit(client_id, client_secret)

In [19]:
resposta = enviar_ordem_teste(token)

Status: 400
Resposta: {'jsonrpc': '2.0', 'error': {'code': 11050, 'message': 'bad_request'}, 'testnet': True, 'usIn': 1744728180084910, 'usOut': 1744728180085003, 'usDiff': 93}


In [20]:
res = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=BTC&kind=future")
dados = res.json()
for i in dados["result"]:
    print(i["instrument_name"])

BTC-18APR25
BTC-25APR25
BTC-30MAY25
BTC-27JUN25
BTC-26SEP25
BTC-26DEC25
BTC-27MAR26
BTC-PERPETUAL


In [21]:
import requests
import json

def enviar_ordem_teste(token):
    # Obter preço atual do BTC-PERPETUAL
    preco_url = "https://test.deribit.com/api/v2/public/ticker?instrument_name=BTC-PERPETUAL"
    preco_resp = requests.get(preco_url).json()
    preco_atual = preco_resp["result"]["last_price"]

    # Configurar requisição
    url = "https://test.deribit.com/api/v2/private/buy"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "instrument_name": "BTC-PERPETUAL",
        "amount": 10,  # Mínimo geralmente é 10 contratos
        "type": "limit",
        "price": preco_atual,  # Preço do momento
        "label": "Calibracao100_teste"
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    print("Status:", response.status_code)
    print("Resposta:", response.json())
    return response.json()

# Executar
resposta = enviar_ordem_teste(token)

Status: 400
Resposta: {'jsonrpc': '2.0', 'error': {'code': 11050, 'message': 'bad_request'}, 'testnet': True, 'usIn': 1744728389109447, 'usOut': 1744728389109472, 'usDiff': 25}


In [22]:
import requests
import json

def enviar_ordem_market(token):
    url = "https://test.deribit.com/api/v2/private/buy"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "instrument_name": "BTC-PERPETUAL",
        "amount": 10,
        "type": "market",
        "label": "Calibracao100_market"
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    print("Status:", response.status_code)
    print("Resposta:", response.json())
    return response.json()

# Executar
resposta = enviar_ordem_market(token)

Status: 400
Resposta: {'jsonrpc': '2.0', 'error': {'code': 11050, 'message': 'bad_request'}, 'testnet': True, 'usIn': 1744728502717825, 'usOut': 1744728502717855, 'usDiff': 30}


In [23]:
def enviar_ordem_limit(token):
    # Obter book para pegar melhor oferta de venda (ask)
    book_url = "https://test.deribit.com/api/v2/public/get_order_book?instrument_name=BTC-PERPETUAL"
    book_res = requests.get(book_url).json()
    best_ask = book_res["result"]["best_ask_price"]
    preco_alvo = best_ask + 10  # Aumenta um pouco para garantir execução

    url = "https://test.deribit.com/api/v2/private/buy"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "instrument_name": "BTC-PERPETUAL",
        "amount": 10,
        "type": "limit",
        "price": preco_alvo,
        "label": "Calibracao100_limit"
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    print("Status:", response.status_code)
    print("Resposta:", response.json())
    return response.json()

# Executar
resposta = enviar_ordem_limit(token)

Status: 400
Resposta: {'jsonrpc': '2.0', 'error': {'code': 11050, 'message': 'bad_request'}, 'testnet': True, 'usIn': 1744728616272196, 'usOut': 1744728616272230, 'usDiff': 34}


In [24]:
import requests
import json

def teste_ordem_debug(token):
    # Pega o preço atual do livro de ofertas
    res = requests.get("https://test.deribit.com/api/v2/public/get_order_book?instrument_name=BTC-PERPETUAL")
    best_ask = res.json()["result"]["best_ask_price"]
    preco_alvo = best_ask + 10

    url = "https://test.deribit.com/api/v2/private/buy"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "instrument_name": "BTC-PERPETUAL",
        "amount": 10,
        "type": "limit",
        "price": preco_alvo,
        "label": "teste_debug_ordem"
    }

    print("Payload enviado:")
    print(json.dumps(payload, indent=2))

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    print("\nStatus:", response.status_code)
    print("Resposta bruta:", response.text)

# Executar
teste_ordem_debug(token)



Payload enviado:
{
  "instrument_name": "BTC-PERPETUAL",
  "amount": 10,
  "type": "limit",
  "price": 85249.0,
  "label": "teste_debug_ordem"
}

Status: 400
Resposta bruta: {"jsonrpc":"2.0","error":{"code":11050,"message":"bad_request"},"testnet":true,"usIn":1744728923504898,"usOut":1744728923504937,"usDiff":39}


In [25]:
def enviar_ordem_limit_com_float(token):
    res = requests.get("https://test.deribit.com/api/v2/public/get_order_book?instrument_name=BTC-PERPETUAL")
    best_ask = res.json()["result"]["best_ask_price"]
    preco_alvo = best_ask + 10

    url = "https://test.deribit.com/api/v2/private/buy"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "instrument_name": "BTC-PERPETUAL",
        "amount": 10.0,  # <<< agora como float
        "type": "limit",
        "price": preco_alvo,
        "label": "teste_float_amount"
    }

    print("Payload enviado:")
    print(json.dumps(payload, indent=2))

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    print("\nStatus:", response.status_code)
    print("Resposta bruta:", response.text)

# Executar
enviar_ordem_limit_com_float(token)

Payload enviado:
{
  "instrument_name": "BTC-PERPETUAL",
  "amount": 10.0,
  "type": "limit",
  "price": 85175.5,
  "label": "teste_float_amount"
}

Status: 400
Resposta bruta: {"jsonrpc":"2.0","error":{"code":11050,"message":"bad_request"},"testnet":true,"usIn":1744729279038841,"usOut":1744729279038873,"usDiff":32}


In [26]:
def enviar_ordem_em_futuro(token):
    res = requests.get("https://test.deribit.com/api/v2/public/get_order_book?instrument_name=BTC-25APR25")
    best_ask = res.json()["result"]["best_ask_price"]
    preco_alvo = best_ask + 10

    url = "https://test.deribit.com/api/v2/private/buy"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "instrument_name": "BTC-25APR25",
        "amount": 10.0,
        "type": "limit",
        "price": preco_alvo,
        "label": "teste_futuro_apr25"
    }

    print("Payload enviado:")
    print(json.dumps(payload, indent=2))

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    print("\nStatus:", response.status_code)
    print("Resposta bruta:", response.text)

# Executar
enviar_ordem_em_futuro(token)

Payload enviado:
{
  "instrument_name": "BTC-25APR25",
  "amount": 10.0,
  "type": "limit",
  "price": 85190.0,
  "label": "teste_futuro_apr25"
}

Status: 400
Resposta bruta: {"jsonrpc":"2.0","error":{"code":11050,"message":"bad_request"},"testnet":true,"usIn":1744729397999548,"usOut":1744729397999583,"usDiff":35}


In [27]:
import json
import requests

def enviar_ordem_em_futuro(token):
    res = requests.get("https://test.deribit.com/api/v2/public/get_order_book?instrument_name=BTC-25APR25")
    best_ask = res.json()["result"]["best_ask_price"]
    preco_alvo = best_ask + 10

    url = "https://test.deribit.com/api/v2/private/buy"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "instrument_name": "BTC-25APR25",
        "amount": 10.0,
        "type": "limit",
        "price": preco_alvo,
        "label": "teste_futuro_apr25"
    }

    print("Payload enviado:")
    print(json.dumps(payload, indent=2))

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    print("\nStatus:", response.status_code)
    print("Resposta bruta:", response.text)

# Executar
enviar_ordem_em_futuro(token)


Payload enviado:
{
  "instrument_name": "BTC-25APR25",
  "amount": 10.0,
  "type": "limit",
  "price": 85357.5,
  "label": "teste_futuro_apr25"
}

Status: 400
Resposta bruta: {"jsonrpc":"2.0","error":{"code":11050,"message":"bad_request"},"testnet":true,"usIn":1744729509600558,"usOut":1744729509600583,"usDiff":25}


In [28]:
from dotenv import load_dotenv
import os

# Caminho correto do seu .env no Google Drive
load_dotenv(dotenv_path="/content/drive/MyDrive/Segredos/.env")

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")

print("client_id:", client_id)
print("client_secret:", client_secret)

client_id: QQX5b-hZ
client_secret: 7rOhlA2OaGCnLf_Y7nz6a4buz_jvkuCNC4Hu0pkfOyU


In [29]:
with open("/content/drive/MyDrive/Segredos/.env", "w") as f:
    f.write("""DERIBIT_CLIENT_ID=ClTMEIjk
DERIBIT_CLIENT_SECRET=PrNenS-YnZUVKPfF1_7bQQ7mkTlbPBnH0PsnJVfxs7E""")

print("✅ Novas credenciais salvas no .env!")

✅ Novas credenciais salvas no .env!


In [30]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="/content/drive/MyDrive/Segredos/.env")

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")

print("client_id:", client_id)
print("client_secret:", client_secret)

client_id: QQX5b-hZ
client_secret: 7rOhlA2OaGCnLf_Y7nz6a4buz_jvkuCNC4Hu0pkfOyU


In [31]:
os.environ.pop("DERIBIT_CLIENT_ID", None)
os.environ.pop("DERIBIT_CLIENT_SECRET", None)

'7rOhlA2OaGCnLf_Y7nz6a4buz_jvkuCNC4Hu0pkfOyU'

In [32]:
# Forçar substituição das variáveis no ambiente
os.environ["DERIBIT_CLIENT_ID"] = "ClTMEIjk"
os.environ["DERIBIT_CLIENT_SECRET"] = "PrNenS-YnZUVKPfF1_7bQQ7mkTlbPBnH0PsnJVfxs7E"

# Carregar para as variáveis do script
client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")

print("✅ Credenciais substituídas manualmente:")
print("client_id:", client_id)
print("client_secret:", client_secret)

✅ Credenciais substituídas manualmente:
client_id: ClTMEIjk
client_secret: PrNenS-YnZUVKPfF1_7bQQ7mkTlbPBnH0PsnJVfxs7E


In [33]:
token = conectar_deribit(client_id, client_secret)

In [34]:
enviar_ordem_em_futuro(token)

Payload enviado:
{
  "instrument_name": "BTC-25APR25",
  "amount": 10.0,
  "type": "limit",
  "price": 85330.0,
  "label": "teste_futuro_apr25"
}

Status: 400
Resposta bruta: {"jsonrpc":"2.0","error":{"code":11050,"message":"bad_request"},"testnet":true,"usIn":1744730991862780,"usOut":1744730991862817,"usDiff":37}


In [35]:
def verificar_escopo(token):
    url = "https://test.deribit.com/api/v2/private/get_account_summary"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "currency": "BTC"
    }

    response = requests.get(url, headers=headers, params=params)
    print("Status:", response.status_code)
    print("Resposta:", response.json())

verificar_escopo(token)

Status: 200
Resposta: {'jsonrpc': '2.0', 'result': {'estimated_liquidation_ratio_map': {'btc_usd': 0.0}, 'available_funds': 99.99907, 'futures_session_upl': 0.0, 'projected_initial_margin': 0.0, 'session_upl': -9.978e-05, 'options_theta': -22.94968, 'options_pl': -9.978e-05, 'equity': 99.99987023, 'change_margin_model_api_limit': {'timeframe': 86400000, 'rate': 10}, 'delta_total': 0.03015, 'fee_balance': 0.0, 'options_gamma': 1e-05, 'limits': {'matching_engine': {'block_rfq_maker': {'rate': 10, 'burst': 20}, 'cancel_all': {'rate': 5, 'burst': 20}, 'guaranteed_mass_quotes': {'rate': 2, 'burst': 2}, 'maximum_mass_quotes': {'rate': 10, 'burst': 10}, 'maximum_quotes': {'rate': 500, 'burst': 500}, 'spot': {'rate': 5, 'burst': 20}, 'trading': {'total': {'rate': 5, 'burst': 20}}}, 'limits_per_currency': False, 'non_matching_engine': {'rate': 20, 'burst': 100}}, 'futures_session_rpl': 0.0, 'currency': 'BTC', 'futures_pl': 0.0, 'margin_balance': 99.99907, 'maintenance_margin': 0.0, 'options_veg

In [36]:
def enviar_ordem_may25(token):
    res = requests.get("https://test.deribit.com/api/v2/public/get_order_book?instrument_name=BTC-30MAY25")
    best_ask = res.json()["result"]["best_ask_price"]
    preco_alvo = best_ask + 10

    url = "https://test.deribit.com/api/v2/private/buy"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "instrument_name": "BTC-30MAY25",
        "amount": 10.0,
        "type": "limit",
        "price": preco_alvo,
        "label": "teste_may25"
    }

    print("Payload enviado:")
    print(json.dumps(payload, indent=2))

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    print("\nStatus:", response.status_code)
    print("Resposta bruta:", response.text)

enviar_ordem_may25(token)

Payload enviado:
{
  "instrument_name": "BTC-30MAY25",
  "amount": 10.0,
  "type": "limit",
  "price": 85692.5,
  "label": "teste_may25"
}

Status: 400
Resposta bruta: {"jsonrpc":"2.0","error":{"code":11050,"message":"bad_request"},"testnet":true,"usIn":1744731295263225,"usOut":1744731295263257,"usDiff":32}


In [14]:
from dotenv import load_dotenv
import os

load_dotenv()

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")
telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

print("client_id:", client_id)
print("client_secret:", client_secret)

client_id: None
client_secret: None


In [12]:
token = conectar_deribit(client_id, client_secret)  # Garante que o token está definido

Exception: Não foi possível conectar à Deribit TESTNET.

In [13]:
print("client_id:", client_id)
print("client_secret:", client_secret)

client_id: None
client_secret: None


In [8]:

import os import time import datetime import requests import threading from dotenv import load_dotenv

=== Carregar variáveis de ambiente ===

load_dotenv() telegram_token = os.getenv("TELEGRAM_BOT_TOKEN") telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID") client_id = os.getenv("DERIBIT_CLIENT_ID") client_secret = os.getenv("DERIBIT_CLIENT_SECRET") estrategia_ativa = "X-Fibra-V2"

=== Função de envio para o Telegram ===

def enviar_alerta_telegram(mensagem): url = f"https://api.telegram.org/bot{telegram_token}/sendMessage" try: requests.post(url, data={"chat_id": telegram_chat_id, "text": mensagem}) except: pass

=== Heartbeat automático a cada 30 minutos ===

def heartbeat(): while True: agora = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") mensagem = ( f"[HEARTBEAT - Calibração 100]\n" f"Sistema ainda ativo\n" f"Horário: {agora}\n" f"Estratégia: {estrategia_ativa}\n" f"Ambiente: TESTNET" ) url = f"https://api.telegram.org/bot{telegram_token}/sendMessage" try: requests.post(url, data={"chat_id": telegram_chat_id, "text": mensagem}) except: pass time.sleep(1800)  # 30 minutos

Iniciar o heartbeat em segundo plano

threading.Thread(target=heartbeat, daemon=True).start()

=== Conexão com a Deribit com log de reconexão ===

def conectar_deribit(client_id, client_secret, max_tentativas=5, tempo_espera_inicial=5): base_url = "https://test.deribit.com/api/v2/public/auth" params = {"grant_type": "client_credentials", "client_id": client_id, "client_secret": client_secret} tentativas = 0 tempo_espera = tempo_espera_inicial

while tentativas < max_tentativas:
    try:
        response = requests.get(base_url, params=params)
        data = response.json()

        if "error" in data:
            with open("log_reconexao.txt", "a") as log:
                log.write(f"[{datetime.datetime.now()}] FALHA: {data['error']['message']}\n")
            mensagem = (
                f"[ALERTA - FALHA DE CONEXÃO]\nHorário: {datetime.datetime.now()}\n"
                f"Erro: {data['error']['message']}\nNova tentativa em {tempo_espera:.1f}s"
            )
            enviar_alerta_telegram(mensagem)
            time.sleep(tempo_espera)
            tentativas += 1
            tempo_espera *= 1.5

        elif "result" in data:
            with open("log_reconexao.txt", "a") as log:
                log.write(f"[{datetime.datetime.now()}] RECONEXÃO ESTABELECIDA\n")
            mensagem = (
                f"[RECONEXÃO ESTABELECIDA]\nHorário: {datetime.datetime.now()}\n"
                f"Ambiente: TESTNET\nEstratégia ativa: {estrategia_ativa}\nStatus: Online"
            )
            enviar_alerta_telegram(mensagem)
            return data['result']['access_token']

    except Exception as e:
        with open("log_reconexao.txt", "a") as log:
            log.write(f"[{datetime.datetime.now()}] EXCEÇÃO: {e}\n")
        enviar_alerta_telegram(f"[EXCEÇÃO] {e}")
        time.sleep(tempo_espera)
        tentativas += 1
        tempo_espera *= 1.5

erro_final = "Não foi possível conectar à Deribit TESTNET."
enviar_alerta_telegram(erro_final)
raise Exception(erro_final)

=== Executar reconexão ===

if client_id and client_secret: token = conectar_deribit(client_id, client_secret) else: enviar_alerta_telegram("[FALHA] Credenciais DERIBIT ausentes ou inválidas. Reconexão abortada.")

SyntaxError: invalid syntax (<ipython-input-8-6505bc16427a>, line 1)

In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

In [9]:


import os import time import datetime import requests import threading from dotenv import load_dotenv

=== Carregar variáveis de ambiente ===

load_dotenv() telegram_token = os.getenv("TELEGRAM_BOT_TOKEN") telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID") client_id = os.getenv("DERIBIT_CLIENT_ID") client_secret = os.getenv("DERIBIT_CLIENT_SECRET") estrategia_ativa = "X-Fibra-V2"

=== Função de envio para o Telegram ===

def enviar_alerta_telegram(mensagem): url = f"https://api.telegram.org/bot{telegram_token}/sendMessage" try: requests.post(url, data={"chat_id": telegram_chat_id, "text": mensagem}) except: pass

=== Heartbeat automático a cada 30 minutos ===

def heartbeat(): while True: agora = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") mensagem = ( f"[HEARTBEAT - Calibração 100]\n" f"Sistema ainda ativo\n" f"Horário: {agora}\n" f"Estratégia: {estrategia_ativa}\n" f"Ambiente: TESTNET" ) url = f"https://api.telegram.org/bot{telegram_token}/sendMessage" try: requests.post(url, data={"chat_id": telegram_chat_id, "text": mensagem}) except: pass time.sleep(1800)  # 30 minutos

Iniciar o heartbeat em segundo plano

threading.Thread(target=heartbeat, daemon=True).start()

=== Conexão com a Deribit com log de reconexão ===

def conectar_deribit(client_id, client_secret, max_tentativas=5, tempo_espera_inicial=5): base_url = "https://test.deribit.com/api/v2/public/auth" params = {"grant_type": "client_credentials", "client_id": client_id, "client_secret": client_secret} tentativas = 0 tempo_espera = tempo_espera_inicial

while tentativas < max_tentativas:
    try:
        response = requests.get(base_url, params=params)
        data = response.json()

        if "error" in data:
            with open("log_reconexao.txt", "a") as log:
                log.write(f"[{datetime.datetime.now()}] FALHA: {data['error']['message']}\n")
            mensagem = (
                f"[ALERTA - FALHA DE CONEXÃO]\nHorário: {datetime.datetime.now()}\n"
                f"Erro: {data['error']['message']}\nNova tentativa em {tempo_espera:.1f}s"
            )
            enviar_alerta_telegram(mensagem)
            time.sleep(tempo_espera)
            tentativas += 1
            tempo_espera *= 1.5

        elif "result" in data:
            with open("log_reconexao.txt", "a") as log:
                log.write(f"[{datetime.datetime.now()}] RECONEXÃO ESTABELECIDA\n")
            mensagem = (
                f"[RECONEXÃO ESTABELECIDA]\nHorário: {datetime.datetime.now()}\n"
                f"Ambiente: TESTNET\nEstratégia ativa: {estrategia_ativa}\nStatus: Online"
            )
            enviar_alerta_telegram(mensagem)
            return data['result']['access_token']

    except Exception as e:
        with open("log_reconexao.txt", "a") as log:
            log.write(f"[{datetime.datetime.now()}] EXCEÇÃO: {e}\n")
        enviar_alerta_telegram(f"[EXCEÇÃO] {e}")
        time.sleep(tempo_espera)
        tentativas += 1
        tempo_espera *= 1.5

erro_final = "Não foi possível conectar à Deribit TESTNET."
enviar_alerta_telegram(erro_final)
raise Exception(erro_final)

=== Executar reconexão ===

if client_id and client_secret: token = conectar_deribit(client_id, client_secret) else: enviar_alerta_telegram("[FALHA] Credenciais DERIBIT ausentes ou inválidas. Reconexão abortada.")

SyntaxError: invalid syntax (<ipython-input-9-6505bc16427a>, line 1)

In [10]:
import os
import time
import datetime
import requests
import threading
from dotenv import load_dotenv

# === Carregar variáveis do .env ===
load_dotenv()
telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")
client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")
estrategia_ativa = "X-Fibra-V2"

def enviar_alerta_telegram(mensagem):
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    try:
        requests.post(url, data={"chat_id": telegram_chat_id, "text": mensagem})
    except:
        pass

def heartbeat():
    while True:
        agora = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        mensagem = (
            f"[HEARTBEAT - Calibração 100]\n"
            f"Sistema ainda ativo\n"
            f"Horário: {agora}\n"
            f"Estratégia: {estrategia_ativa}\n"
            f"Ambiente: TESTNET"
        )
        url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
        try:
            requests.post(url, data={"chat_id": telegram_chat_id, "text": mensagem})
        except:
            pass
        time.sleep(1800)

threading.Thread(target=heartbeat, daemon=True).start()

def conectar_deribit(client_id, client_secret, max_tentativas=5, tempo_espera_inicial=5):
    base_url = "https://test.deribit.com/api/v2/public/auth"
    params = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    tentativas = 0
    tempo_espera = tempo_espera_inicial

    while tentativas < max_tentativas:
        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "error" in data:
                with open("log_reconexao.txt", "a") as log:
                    log.write(f"[{datetime.datetime.now()}] FALHA: {data['error']['message']}\n")
                mensagem = (
                    f"[ALERTA - FALHA DE CONEXÃO]\nHorário: {datetime.datetime.now()}\n"
                    f"Erro: {data['error']['message']}\nNova tentativa em {tempo_espera:.1f}s"
                )
                enviar_alerta_telegram(mensagem)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5

            elif "result" in data:
                with open("log_reconexao.txt", "a") as log:
                    log.write(f"[{datetime.datetime.now()}] RECONEXÃO ESTABELECIDA\n")
                mensagem = (
                    f"[RECONEXÃO ESTABELECIDA]\nHorário: {datetime.datetime.now()}\n"
                    f"Ambiente: TESTNET\nEstratégia ativa: {estrategia_ativa}\nStatus: Online"
                )
                enviar_alerta_telegram(mensagem)
                return data['result']['access_token']

        except Exception as e:
            with open("log_reconexao.txt", "a") as log:
                log.write(f"[{datetime.datetime.now()}] EXCEÇÃO: {e}\n")
            enviar_alerta_telegram(f"[EXCEÇÃO] {e}")
            time.sleep(tempo_espera)
            tentativas += 1
            tempo_espera *= 1.5

    erro_final = "Não foi possível conectar à Deribit TESTNET."
    enviar_alerta_telegram(erro_final)
    raise Exception(erro_final)

# === Iniciar conexão ===
if client_id and client_secret:
    token = conectar_deribit(client_id, client_secret)
else:
    enviar_alerta_telegram("[FALHA] Credenciais DERIBIT ausentes ou inválidas.")

In [11]:
import json

def enviar_ordem_teste(token):
    url = "https://test.deribit.com/api/v2/private/buy"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "instrument_name": "BTC-PERPETUAL",
        "amount": 10,
        "type": "limit",
        "price": 25000,  # Ajuste para preço aceitável atual
        "label": "Calibracao100_teste"
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    print("Status:", response.status_code)
    print("Resposta:", response.json())
    return response.json()

# Executar
resposta = enviar_ordem_teste(token)

NameError: name 'token' is not defined

In [ ]:
# === Calibração 100 | Inicialização ===
import time
import datetime
import threading

def loop_de_manutencao(duracao_em_horas=12):
    def manter_ativo():
        tempo_total = duracao_em_horas * 3600
        inicio = time.time()
        print(f"[Manutenção] Ativa por {duracao_em_horas} horas.")

        while True:
            agora = time.time()
            if agora - inicio >= tempo_total:
                print("[Manutenção] Finalizada.")
                break
            print(f"[{datetime.datetime.now().strftime('%H:%M:%S')}] Sistema ainda ativo...")
            time.sleep(600)  # Espera 10 minutos

    thread = threading.Thread(target=manter_ativo)
    thread.daemon = True
    thread.start()

# Inicia a manutenção assim que o notebook for rodado
loop_de_manutencao(duracao_em_horas=12)

print("Calibração 100 iniciado com manutenção ativa.")

[Manutenção] Ativa por 12 horas.Calibração 100 iniciado com manutenção ativa.

[05:34:53] Sistema ainda ativo...


In [ ]:
import requests
import time
import datetime

def conectar_deribit(client_id, client_secret, max_tentativas=5, tempo_espera_inicial=5):
    base_url = "https://test.deribit.com/api/v2/public/auth"
    params = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    tentativas = 0
    tempo_espera = tempo_espera_inicial

    while tentativas < max_tentativas:
        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "error" in data:
                codigo_erro = data["error"].get("code", "N/A")
                mensagem = data["error"].get("message", "Erro desconhecido")
                agora = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                log = (
                    "[ALERTA - FALHA DE CONEXÃO]\n"
                    f"Horário: {agora}\n"
                    f"Erro: deribit {data}\n"
                    "O robô tentará reconectar automaticamente."
                )
                print("Calibração 100:", log)
                enviar_alerta_telegram(log)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5

            elif "result" in data:
                agora = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                msg_reconexao = (
                    "[RECONEXÃO ESTABELECIDA]\n"
                    f"Horário: {agora}\n"
                    f"Ambiente: TESTNET\n"
                    f"Estratégia ativa: {estrategia_ativa}\n"
                    f"Status: Online"
                )
                print("Calibração 100:", msg_reconexao)
                enviar_alerta_telegram(msg_reconexao)
                return data["result"]["access_token"]

            else:
                msg = f"[{tentativas+1}] Erro inesperado: {data}"
                print("Calibração 100:", msg)
                enviar_alerta_telegram(msg)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5

        except Exception as e:
            msg = f"[{tentativas+1}] Erro de exceção: {e}"
            print("Calibração 100:", msg)
            enviar_alerta_telegram(msg)
            time.sleep(tempo_espera)
            tentativas += 1
            tempo_espera *= 1.5

    erro_final = "Não foi possível conectar à Deribit TESTNET após várias tentativas."
    print("Calibração 100:", erro_final)
    enviar_alerta_telegram(erro_final)
    raise Exception(erro_final)

In [ ]:
token = conectar_deribit(client_id, client_secret)

Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:40:33
Erro: deribit {'jsonrpc': '2.0', 'error': {'code': 13004, 'message': 'invalid_credentials'}, 'testnet': True, 'usIn': 1744695633018160, 'usOut': 1744695633018268, 'usDiff': 108}
O robô tentará reconectar automaticamente.
Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:40:39
Erro: deribit {'jsonrpc': '2.0', 'error': {'code': 13004, 'message': 'invalid_credentials'}, 'testnet': True, 'usIn': 1744695638957427, 'usOut': 1744695638957489, 'usDiff': 62}
O robô tentará reconectar automaticamente.
Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:40:47
Erro: deribit {'jsonrpc': '2.0', 'error': {'code': 13004, 'message': 'invalid_credentials'}, 'testnet': True, 'usIn': 1744695647391193, 'usOut': 1744695647391265, 'usDiff': 72}
O robô tentará reconectar automaticamente.
Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:40:59
Erro: deribit {'jsonrpc': '2.0', 'error': {'cod

Exception: Não foi possível conectar à Deribit TESTNET após várias tentativas.

In [ ]:
# Verificar se as variáveis foram carregadas corretamente
def verificar_credenciais():
    if not client_id or not client_secret:
        print("❌ As credenciais DERIBIT não foram carregadas corretamente!")
        print("Verifique seu .env ou recarregue com load_dotenv().")
        return False
    print("✅ Credenciais DERIBIT carregadas com sucesso.")
    return True

# Rodar a verificação e tentar conexão só se estiver tudo certo
if verificar_credenciais():
    token = conectar_deribit(client_id, client_secret)
else:
    enviar_alerta_telegram("[FALHA] Credenciais DERIBIT ausentes ou inválidas. Reconexão abortada.")

✅ Credenciais DERIBIT carregadas com sucesso.
Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:42:39
Erro: deribit {'jsonrpc': '2.0', 'error': {'code': 13004, 'message': 'invalid_credentials'}, 'testnet': True, 'usIn': 1744695759268146, 'usOut': 1744695759268202, 'usDiff': 56}
O robô tentará reconectar automaticamente.
Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:42:45
Erro: deribit {'jsonrpc': '2.0', 'error': {'code': 13004, 'message': 'invalid_credentials'}, 'testnet': True, 'usIn': 1744695765198417, 'usOut': 1744695765198482, 'usDiff': 65}
O robô tentará reconectar automaticamente.
Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:42:53
Erro: deribit {'jsonrpc': '2.0', 'error': {'code': 13004, 'message': 'invalid_credentials'}, 'testnet': True, 'usIn': 1744695773654021, 'usOut': 1744695773654089, 'usDiff': 68}
O robô tentará reconectar automaticamente.
Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:43:05
Er

Exception: Não foi possível conectar à Deribit TESTNET após várias tentativas.

In [ ]:
!cat /content/drive/MyDrive/Segredos/.env

DERIBIT_CLIENT_ID=w83RPW8w
DERIBIT_CLIENT_SECRET=EfCgKLcX0VsnMpvdGbunv8Pz0CbTAzS7K3H45b6q_YA
TELEGRAM_BOT_TOKEN=7589434345:AAF-k9jl7JLAy5I2rgaN1JTJDGd7Dnx7D9c
TELEGRAM_CHAT_ID=5934484407

In [ ]:
response = requests.get(
    "https://test.deribit.com/api/v2/public/auth",
    params={
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
)
print("Status:", response.status_code)
print("Resposta:", response.json())


Status: 400
Resposta: {'jsonrpc': '2.0', 'error': {'code': 13004, 'message': 'invalid_credentials'}, 'testnet': True, 'usIn': 1744696043236089, 'usOut': 1744696043236164, 'usDiff': 75}


In [ ]:
env_path = "/content/drive/MyDrive/Segredos/.env"

conteudo = """DERIBIT_CLIENT_ID=QQX5b-hZ
DERIBIT_CLIENT_SECRET=7rOhlA2OaGCnLf_Y7nz6a4buz_jvkuCNC4Hu0pkfOyU
TELEGRAM_BOT_TOKEN=7589434345:AAF-k9jl7JLAy5I2rgaN1JTJDGd7Dnx7D9c
TELEGRAM_CHAT_ID=5934484407
"""

with open(env_path, "w") as f:
    f.write(conteudo.strip())

print("✅ Novo .env salvo com sucesso!")

✅ Novo .env salvo com sucesso!


In [ ]:
from dotenv import load_dotenv
load_dotenv("/content/drive/MyDrive/Segredos/.env")

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")

print("client_id:", client_id)

client_id: QHy4D9s-


In [ ]:
token = conectar_deribit(client_id, client_secret)

Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:53:28
Erro: deribit {'jsonrpc': '2.0', 'error': {'code': 13004, 'message': 'invalid_credentials'}, 'testnet': True, 'usIn': 1744696408019905, 'usOut': 1744696408019965, 'usDiff': 60}
O robô tentará reconectar automaticamente.
Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:53:34
Erro: deribit {'jsonrpc': '2.0', 'error': {'code': 13004, 'message': 'invalid_credentials'}, 'testnet': True, 'usIn': 1744696414081764, 'usOut': 1744696414081835, 'usDiff': 71}
O robô tentará reconectar automaticamente.
Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:53:42
Erro: deribit {'jsonrpc': '2.0', 'error': {'code': 13004, 'message': 'invalid_credentials'}, 'testnet': True, 'usIn': 1744696422507063, 'usOut': 1744696422507130, 'usDiff': 67}
O robô tentará reconectar automaticamente.
Calibração 100: [ALERTA - FALHA DE CONEXÃO]
Horário: 2025-04-15 05:53:54
Erro: deribit {'jsonrpc': '2.0', 'error': {'code

Exception: Não foi possível conectar à Deribit TESTNET após várias tentativas.

In [ ]:
import requests

response = requests.get(
    "https://test.deribit.com/api/v2/public/auth",
    params={
        "grant_type": "client_credentials",
        "client_id": "QQX5b-hZ",
        "client_secret": "7rOhlA2OaGCnLf_Y7nz6a4buz_jvkuCNC4Hu0pkfOyU"
    }
)

print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: {'jsonrpc': '2.0', 'result': {'enabled_features': [], 'access_token': '1744697399227.1TL9G1Xs.H499ZzpoCbNr-bvDUJaEEW8KgqeHxZoObH90fO_Ir5daIlu2IZR-YQsy0agFikiomOvzI3F8JicwtYhosFiP1R1EmGcNB_uuR-mv11A3p0Z9dhiYiZMXBZCUgaiW9V5U11mMAMCRMM_nx7EIqg84CXeu7qGsvgeSUQKs1umra9yUxedr4lzEUnjH85SpQVnhTMYU14wJfc9r8vosPCvGSnddvj-tzSYGgzUE5nUfKkrTxV9n-O855QQZod2DCcNC4CQ', 'expires_in': 900, 'refresh_token': '1745301299227.1bpB_zz8.8ExJRXENQYCBrccwwrx0VeWcHt0mdCWNymjIER6DTzyJ3YujJ5X5f_48kkNY-srhxDZkODZw0-zT2kk07hIa-I_1QVKht864Ls91QmA0V_J503ocAdXFSCaGXQ-ME8O8B5x5ZbgbQTivlFLaOs1krEZdonlMIdXqb8aQzcQIN-Y592OMEFteWd_XwRyDH7KlpDzzU6qtwrB6s6d8OMN5NJ_XuVcWNJKx9xJXqoVX0lnMxIXNRmOUOJSMcKXLhsDKSA', 'scope': 'session:rest-C3k/akvVhHs= trade:read_write account:read wallet:read mainaccount', 'sid': '67378.QQX5b-hZ.rest-C3k/akvVhHs=', 'token_type': 'bearer'}, 'usIn': 1744696499227309, 'usOut': 1744696499227681, 'usDiff': 372, 'testnet': True}


In [ ]:

import os
from dotenv import load_dotenv

# Limpa variáveis antigas da memória
os.environ.pop("DERIBIT_CLIENT_ID", None)
os.environ.pop("DERIBIT_CLIENT_SECRET", None)

# Recarrega do .env atualizado
load_dotenv("/content/drive/MyDrive/Segredos/.env")

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")

print("client_id:", client_id)

client_id: QQX5b-hZ


In [ ]:
token = conectar_deribit(client_id, client_secret)

Calibração 100: [RECONEXÃO ESTABELECIDA]
Horário: 2025-04-15 05:57:39
Ambiente: TESTNET
Estratégia ativa: X-Fibra-V2
Status: Online


In [1]:
from datetime import datetime
import os
import requests
from dotenv import load_dotenv

load_dotenv()

telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

def enviar_teste():
    agora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    mensagem = (
        f"[Calibração 100 - TESTE DE STATUS]\n"
        f"Status: Sistema ativo\n"
        f"Horário: {agora}\n"
        f"Estratégia: X-Fibra-V2 (teste)\n"
        f"Ambiente: TESTNET\n"
    )
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    response = requests.post(url, data={"chat_id": telegram_chat_id, "text": mensagem})
    print("Status do envio:", response.status_code)

enviar_teste()

ModuleNotFoundError: No module named 'dotenv'

In [2]:
!pip install python-dotenv


In [3]:
from datetime import datetime
import os
import requests
from dotenv import load_dotenv

load_dotenv()

telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

def enviar_teste():
    agora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    mensagem = (
        f"[Calibração 100 - TESTE DE STATUS]\n"
        f"Status: Sistema ativo\n"
        f"Horário: {agora}\n"
        f"Estratégia: X-Fibra-V2 (teste)\n"
        f"Ambiente: TESTNET\n"
    )
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    response = requests.post(url, data={"chat_id": telegram_chat_id, "text": mensagem})
    print("Status do envio:", response.status_code)

enviar_teste()

Status do envio: 404


In [4]:
import requests
from datetime import datetime

# Substitua aqui pelos seus dados reais:
telegram_token = "7589434345:AAF-k9jl7JLAy5I2rgaN1JTJDGd7Dnx7D9c"
telegram_chat_id = "5934484407"

def enviar_teste_direto():
    agora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    mensagem = (
        f"[Calibração 100 - TESTE DE STATUS]\n"
        f"Status: Sistema ativo\n"
        f"Horário: {agora}\n"
        f"Estratégia: X-Fibra-V2 (teste)\n"
        f"Ambiente: TESTNET\n"
    )
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    response = requests.post(url, data={"chat_id": telegram_chat_id, "text": mensagem})
    print("Status do envio:", response.status_code)
    print("Resposta:", response.json())

enviar_teste_direto()

Status do envio: 200
Resposta: {'ok': True, 'result': {'message_id': 554, 'from': {'id': 7589434345, 'is_bot': True, 'first_name': 'Calibração 100', 'username': 'QCriptoBot'}, 'chat': {'id': 5934484407, 'first_name': 'Jackson', 'last_name': 'Araujo', 'type': 'private'}, 'date': 1744725678, 'text': '[Calibração 100 - TESTE DE STATUS]\nStatus: Sistema ativo\nHorário: 2025-04-15 14:01:18\nEstratégia: X-Fibra-V2 (teste)\nAmbiente: TESTNET'}}


In [5]:
import threading

def heartbeat():
    while True:
        agora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        mensagem = (
            f"[HEARTBEAT - Calibração 100]\n"
            f"Sistema ainda ativo\n"
            f"Horário: {agora}\n"
            f"Estratégia: {estrategia_ativa}\n"
            f"Ambiente: TESTNET"
        )
        url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
        try:
            requests.post(url, data={"chat_id": telegram_chat_id, "text": mensagem})
        except:
            pass
        time.sleep(1800)  # 30 minutos

# Iniciar heartbeat em thread paralela
threading.Thread(target=heartbeat, daemon=True).start()

Exception in thread Thread-8 (heartbeat):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner


In [ ]:
env_path = "/content/drive/MyDrive/Segredos/.env"

conteudo = """DERIBIT_CLIENT_ID=QHy4D9s-
DERIBIT_CLIENT_SECRET=y47wloxUzjTKRoCkr-9H5oUW8l0sgOdG3yzvE165EIU
TELEGRAM_BOT_TOKEN=7589434345:AAF-k9jl7JLAy5I2rgaN1JTJDGd7Dnx7D9c
TELEGRAM_CHAT_ID=5934484407
"""

with open(env_path, "w") as f:
    f.write(conteudo.strip())

print("Arquivo .env criado com sucesso em:", env_path)

Arquivo .env criado com sucesso em: /content/drive/MyDrive/Segredos/.env


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install -q python-dotenv

import os
from dotenv import load_dotenv

env_path = '/content/drive/MyDrive/Segredos/.env'
load_dotenv(env_path)

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")
telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

if client_id and client_secret and telegram_token and telegram_chat_id:
    print("Chaves carregadas com sucesso!")
else:
    print("Erro: Verifique se todas as chaves estão corretas no .env")

Chaves carregadas com sucesso!


In [ ]:
import requests
import time

def enviar_alerta_telegram(msg):
    if not telegram_token or not telegram_chat_id:
        print("Telegram não configurado.")
        return
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    payload = {
        "chat_id": telegram_chat_id,
        "text": msg
    }
    try:
        requests.post(url, data=payload)
    except Exception as e:
        print("Erro ao enviar alerta:", e)

def conectar_deribit(client_id, client_secret, max_tentativas=5):
    url = "https://www.deribit.com/api/v2/public/auth"
    payload = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    for tentativa in range(max_tentativas):
        try:
            response = requests.post(url, data=payload)
            data = response.json()

            if "error" in data and data["error"]["code"] == 13004:
                msg = f"[{tentativa+1}] Erro: invalid_credentials. Tentando em 5s..."
                print(msg)
                enviar_alerta_telegram(msg)
                time.sleep(5)
            elif "result" in data:
                print("Conexão bem-sucedida com a Deribit!")
                return data["result"]["access_token"]
            else:
                print("Erro inesperado:", data)
                break
        except Exception as e:
            print(f"[{tentativa+1}] Erro ao conectar: {e}")
            enviar_alerta_telegram(f"Erro ao conectar: {e}")
            time.sleep(5)

    msg = "Não foi possível conectar à Deribit após várias tentativas."
    enviar_alerta_telegram(msg)
    raise Exception(msg)

# Executar a função
token = conectar_deribit(client_id, client_secret)

Erro inesperado: {'jsonrpc': '2.0', 'error': {'code': 11050, 'data': 'invalid json', 'message': 'bad_request'}, 'testnet': False, 'usIn': 1744687491115719, 'usOut': 1744687491115788, 'usDiff': 69}


Exception: Não foi possível conectar à Deribit após várias tentativas.

In [ ]:
import requests
import time

def enviar_alerta_telegram(msg):
    if not telegram_token or not telegram_chat_id:
        print("Telegram não configurado.")
        return
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    payload = {
        "chat_id": telegram_chat_id,
        "text": msg
    }
    try:
        requests.post(url, data=payload)
    except Exception as e:
        print("Erro ao enviar alerta:", e)

def conectar_deribit(client_id, client_secret, max_tentativas=5):
    url = "https://www.deribit.com/api/v2/public/auth"
    payload = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    for tentativa in range(max_tentativas):
        try:
            # AQUI ESTÁ A CORREÇÃO: usar json=payload
            response = requests.post(url, json=payload)
            data = response.json()

            if "error" in data and data["error"]["code"] == 13004:
                msg = f"[{tentativa+1}] Erro: invalid_credentials. Tentando em 5s..."
                print(msg)
                enviar_alerta_telegram(msg)
                time.sleep(5)
            elif "result" in data:
                print("Conexão bem-sucedida com a Deribit!")
                return data["result"]["access_token"]
            else:
                print("Erro inesperado:", data)
                break
        except Exception as e:
            print(f"[{tentativa+1}] Erro ao conectar: {e}")
            enviar_alerta_telegram(f"Erro ao conectar: {e}")
            time.sleep(5)

    msg = "Não foi possível conectar à Deribit após várias tentativas."
    enviar_alerta_telegram(msg)
    raise Exception(msg)

# Executar novamente
token = conectar_deribit(client_id, client_secret)

Erro inesperado: {'jsonrpc': '2.0', 'error': {'code': 11050, 'message': 'bad_request'}, 'testnet': False, 'usIn': 1744687577158989, 'usOut': 1744687577159023, 'usDiff': 34}


Exception: Não foi possível conectar à Deribit após várias tentativas.

In [ ]:
import requests
import time

def enviar_alerta_telegram(msg):
    if not telegram_token or not telegram_chat_id:
        print("Telegram não configurado.")
        return
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    payload = {
        "chat_id": telegram_chat_id,
        "text": msg
    }
    try:
        requests.post(url, data=payload)
    except Exception as e:
        print("Erro ao enviar alerta:", e)

def conectar_deribit(client_id, client_secret, max_tentativas=5):
    base_url = "https://www.deribit.com/api/v2/public/auth"
    params = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    for tentativa in range(max_tentativas):
        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "error" in data and data["error"]["code"] == 13004:
                msg = f"[{tentativa+1}] Erro: invalid_credentials. Tentando em 5s..."
                print(msg)
                enviar_alerta_telegram(msg)
                time.sleep(5)
            elif "result" in data:
                print("Conexão bem-sucedida com a Deribit!")
                return data["result"]["access_token"]
            else:
                print("Erro inesperado:", data)
                break
        except Exception as e:
            print(f"[{tentativa+1}] Erro ao conectar: {e}")
            enviar_alerta_telegram(f"Erro ao conectar: {e}")
            time.sleep(5)

    msg = "Não foi possível conectar à Deribit após várias tentativas."
    enviar_alerta_telegram(msg)
    raise Exception(msg)

# Executar
token = conectar_deribit(client_id, client_secret)

[1] Erro: invalid_credentials. Tentando em 5s...
[2] Erro: invalid_credentials. Tentando em 5s...
[3] Erro: invalid_credentials. Tentando em 5s...
[4] Erro: invalid_credentials. Tentando em 5s...
[5] Erro: invalid_credentials. Tentando em 5s...


Exception: Não foi possível conectar à Deribit após várias tentativas.

In [ ]:
import requests
import time

def enviar_alerta_telegram(msg):
    if not telegram_token or not telegram_chat_id:
        print("Telegram não configurado.")
        return
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    payload = {
        "chat_id": telegram_chat_id,
        "text": f"Calibração 100:\n{msg}"
    }
    try:
        requests.post(url, data=payload)
    except Exception as e:
        print("Erro ao enviar alerta:", e)

def conectar_deribit(client_id, client_secret, max_tentativas=5):
    base_url = "https://www.deribit.com/api/v2/public/auth"
    params = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    for tentativa in range(max_tentativas):
        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "error" in data and data["error"]["code"] == 13004:
                msg = f"[{tentativa+1}] Erro: invalid_credentials. Tentando em 5s..."
                print(msg)
                enviar_alerta_telegram(msg)
                time.sleep(5)
            elif "result" in data:
                print("Conexão bem-sucedida com a Deribit!")
                return data["result"]["access_token"]
            else:
                print("Erro inesperado:", data)
                enviar_alerta_telegram(str(data))
                break
        except Exception as e:
            msg = f"[{tentativa+1}] Erro ao conectar: {e}"
            print(msg)
            enviar_alerta_telegram(msg)
            time.sleep(5)

    msg = "Não foi possível conectar à Deribit após várias tentativas."
    enviar_alerta_telegram(msg)
    raise Exception(msg)

# Chamar a função:
token = conectar_deribit(client_id, client_secret)

[1] Erro: invalid_credentials. Tentando em 5s...
[2] Erro: invalid_credentials. Tentando em 5s...
[3] Erro: invalid_credentials. Tentando em 5s...
[4] Erro: invalid_credentials. Tentando em 5s...
[5] Erro: invalid_credentials. Tentando em 5s...


Exception: Não foi possível conectar à Deribit após várias tentativas.

In [ ]:
from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/Segredos/.env')

client_id = os.getenv("_i2pppal")
client_secret = os.getenv("mDJf848VWxaVVSdyZHyjzAVjwllWduHiudReogH5zNA")
telegram_token = os.getenv("7589434345:AAF-k9jl7JLAy5I2rgaN1JTJDGd7Dnx7D9c")
telegram_chat_id = os.getenv("5934484407")

print("Chaves recarregadas.")

Chaves recarregadas.


In [ ]:
import requests
import time

def enviar_alerta_telegram(msg):
    if not telegram_token or not telegram_chat_id:
        print("[Telegram] Não configurado.")
        return
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    payload = {
        "chat_id": telegram_chat_id,
        "text": f"[Calibração 100 - ALERTA]\n{msg}"
    }
    try:
        requests.post(url, data=payload)
    except Exception as e:
        print("Erro ao enviar alerta para o Telegram:", e)

def conectar_deribit(client_id, client_secret, max_tentativas=10, tempo_espera_inicial=5):
    base_url = "https://www.deribit.com/api/v2/public/auth"
    params = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    tentativas = 0
    tempo_espera = tempo_espera_inicial

    while tentativas < max_tentativas:
        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "error" in data:
                codigo_erro = data["error"].get("code", "N/A")
                mensagem = data["error"].get("message", "Erro desconhecido")
                log = f"[{tentativas+1}/{max_tentativas}] Erro {codigo_erro}: {mensagem}. Nova tentativa em {tempo_espera}s..."
                print("Calibração 100:", log)
                enviar_alerta_telegram(log)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5  # espera progressiva
            elif "result" in data:
                print("Conexão bem-sucedida com a Deribit!")
                enviar_alerta_telegram("Conexão bem-sucedida com a Deribit!")
                return data["result"]["access_token"]
            else:
                msg = f"[{tentativas+1}] Erro inesperado: {data}"
                print("Calibração 100:", msg)
                enviar_alerta_telegram(msg)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5
        except Exception as e:
            msg = f"[{tentativas+1}] Erro de exceção: {e}"
            print("Calibração 100:", msg)
            enviar_alerta_telegram(msg)
            time.sleep(tempo_espera)
            tentativas += 1
            tempo_espera *= 1.5

    erro_final = "Não foi possível conectar à Deribit após várias tentativas."
    print("Calibração 100:", erro_final)
    enviar_alerta_telegram(erro_final)
    raise Exception(erro_final)

In [ ]:
print("client_id:", client_id)
print("client_secret:", "*" * len(client_secret) if client_secret else None)
print("telegram_token:", "*" * len(telegram_token) if telegram_token else None)
print("telegram_chat_id:", telegram_chat_id)

client_id: None
client_secret: None
telegram_token: None
telegram_chat_id: None


In [ ]:
import os
print(os.path.exists("/content/drive/MyDrive/Segredos/.env"))

True


In [ ]:
with open("/content/drive/MyDrive/Segredos/.env", "r") as f:
    print(f.read())

DERIBIT_CLIENT_ID=QHy4D9s-
DERIBIT_CLIENT_SECRET=y47wloxUzjTKRoCkr-9H5oUW8l0sgOdG3yzvE165EIU
TELEGRAM_BOT_TOKEN=7589434345:AAF-k9jl7JLAy5I2rgaN1JTJDGd7Dnx7D9c
TELEGRAM_CHAT_ID=5934484407


In [ ]:


from dotenv import load_dotenv
load_dotenv("/content/drive/MyDrive/Segredos/.env")

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")
telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

print("Variáveis carregadas!")

Variáveis carregadas!


In [ ]:
token = conectar_deribit(client_id, client_secret)

Calibração 100: [1/10] Erro 13004: invalid_credentials. Nova tentativa em 5s...
Calibração 100: [2/10] Erro 13004: invalid_credentials. Nova tentativa em 7.5s...
Calibração 100: [3/10] Erro 13004: invalid_credentials. Nova tentativa em 11.25s...
Calibração 100: [4/10] Erro 13004: invalid_credentials. Nova tentativa em 16.875s...
Calibração 100: [5/10] Erro 13004: invalid_credentials. Nova tentativa em 25.3125s...
Calibração 100: [6/10] Erro 13004: invalid_credentials. Nova tentativa em 37.96875s...
Calibração 100: [7/10] Erro 13004: invalid_credentials. Nova tentativa em 56.953125s...
Calibração 100: [8/10] Erro 13004: invalid_credentials. Nova tentativa em 85.4296875s...
Calibração 100: [9/10] Erro 13004: invalid_credentials. Nova tentativa em 128.14453125s...
Calibração 100: [10/10] Erro 13004: invalid_credentials. Nova tentativa em 192.216796875s...
Calibração 100: Não foi possível conectar à Deribit após várias tentativas.


Exception: Não foi possível conectar à Deribit após várias tentativas.

In [ ]:

import requests
import time

def conectar_deribit_testnet(client_id, client_secret, max_tentativas=10, tempo_espera_inicial=5):
    base_url = "https://test.deribit.com/api/v2/public/auth"
    params = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    tentativas = 0
    tempo_espera = tempo_espera_inicial

    while tentativas < max_tentativas:
        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "error" in data:
                codigo_erro = data["error"].get("code", "N/A")
                mensagem = data["error"].get("message", "Erro desconhecido")
                log = f"[{tentativas+1}/{max_tentativas}] Erro {codigo_erro}: {mensagem}. Nova tentativa em {tempo_espera}s..."
                print("Calibração 100:", log)
                enviar_alerta_telegram(log)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5
            elif "result" in data:
                print("Conexão bem-sucedida com a Deribit TESTNET!")
                enviar_alerta_telegram("Conexão bem-sucedida com a Deribit TESTNET!")
                return data["result"]["access_token"]
            else:
                msg = f"[{tentativas+1}] Erro inesperado: {data}"
                print("Calibração 100:", msg)
                enviar_alerta_telegram(msg)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5
        except Exception as e:
            msg = f"[{tentativas+1}] Erro de exceção: {e}"
            print("Calibração 100:", msg)
            enviar_alerta_telegram(msg)
            time.sleep(tempo_espera)
            tentativas += 1
            tempo_espera *= 1.5

    erro_final = "Não foi possível conectar à Deribit TESTNET após várias tentativas."
    print("Calibração 100:", erro_final)
    enviar_alerta_telegram(erro_final)
    raise Exception(erro_final)

In [ ]:
import requests
import time

def enviar_alerta_telegram(msg):
    if not telegram_token or not telegram_chat_id:
        print("[Telegram] Não configurado.")
        return
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    payload = {
        "chat_id": telegram_chat_id,
        "text": f"[Calibração 100 - ALERTA]\n{msg}"
    }
    try:
        requests.post(url, data=payload)
    except Exception as e:
        print("Erro ao enviar alerta para o Telegram:", e)

def conectar_deribit_testnet(client_id, client_secret, max_tentativas=10, tempo_espera_inicial=5):
    base_url = "https://test.deribit.com/api/v2/public/auth"
    params = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    tentativas = 0
    tempo_espera = tempo_espera_inicial

    while tentativas < max_tentativas:
        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "error" in data:
                codigo_erro = data["error"].get("code", "N/A")
                mensagem = data["error"].get("message", "Erro desconhecido")
                log = f"[{tentativas+1}/{max_tentativas}] Erro {codigo_erro}: {mensagem}. Nova tentativa em {tempo_espera}s..."
                print("Calibração 100:", log)
                enviar_alerta_telegram(log)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5
            elif "result" in data:
                print("Conexão bem-sucedida com a Deribit TESTNET!")
                enviar_alerta_telegram("Conexão bem-sucedida com a Deribit TESTNET!")
                return data["result"]["access_token"]
            else:
                msg = f"[{tentativas+1}] Erro inesperado: {data}"
                print("Calibração 100:", msg)
                enviar_alerta_telegram(msg)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5
        except Exception as e:
            msg = f"[{tentativas+1}] Erro de exceção: {e}"
            print("Calibração 100:", msg)
            enviar_alerta_telegram(msg)
            time.sleep(tempo_espera)
            tentativas += 1
            tempo_espera *= 1.5

    erro_final = "Não foi possível conectar à Deribit TESTNET após várias tentativas."
    print("Calibração 100:", erro_final)
    enviar_alerta_telegram(erro_final)
    raise Exception(erro_final)

In [ ]:
token = conectar_deribit_testnet(client_id, client_secret)

Calibração 100: [1/10] Erro 13004: invalid_credentials. Nova tentativa em 5s...
Calibração 100: [2/10] Erro 13004: invalid_credentials. Nova tentativa em 7.5s...
Calibração 100: [3/10] Erro 13004: invalid_credentials. Nova tentativa em 11.25s...
Calibração 100: [4/10] Erro 13004: invalid_credentials. Nova tentativa em 16.875s...
Calibração 100: [5/10] Erro 13004: invalid_credentials. Nova tentativa em 25.3125s...
Calibração 100: [6/10] Erro 13004: invalid_credentials. Nova tentativa em 37.96875s...
Calibração 100: [7/10] Erro 13004: invalid_credentials. Nova tentativa em 56.953125s...
Calibração 100: [8/10] Erro 13004: invalid_credentials. Nova tentativa em 85.4296875s...
Calibração 100: [9/10] Erro 13004: invalid_credentials. Nova tentativa em 128.14453125s...
Calibração 100: [10/10] Erro 13004: invalid_credentials. Nova tentativa em 192.216796875s...


KeyboardInterrupt: 

In [ ]:
env_path = "/content/drive/MyDrive/Segredos/.env"

conteudo = """DERIBIT_CLIENT_ID=w83RPW8w
DERIBIT_CLIENT_SECRET=EfCgKLcX0VsnMpvdGbunv8Pz0CbTAzS7K3H45b6q_YA
TELEGRAM_BOT_TOKEN=7589434345:AAF-k9jl7JLAy5I2rgaN1JTJDGd7Dnx7D9c
TELEGRAM_CHAT_ID=5934484407
"""

with open(env_path, "w") as f:
    f.write(conteudo.strip())

print("Arquivo .env atualizado com as novas chaves!")

Arquivo .env atualizado com as novas chaves!


In [ ]:
from dotenv import load_dotenv
load_dotenv("/content/drive/MyDrive/Segredos/.env")

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")
telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

print("Variáveis atualizadas!")

Variáveis atualizadas!


In [ ]:
token = conectar_deribit_testnet(client_id, client_secret)

Calibração 100: [1/10] Erro 13004: invalid_credentials. Nova tentativa em 5s...
Calibração 100: [2/10] Erro 13004: invalid_credentials. Nova tentativa em 7.5s...
Calibração 100: [3/10] Erro 13004: invalid_credentials. Nova tentativa em 11.25s...
Calibração 100: [4/10] Erro 13004: invalid_credentials. Nova tentativa em 16.875s...
Calibração 100: [5/10] Erro 13004: invalid_credentials. Nova tentativa em 25.3125s...
Calibração 100: [6/10] Erro 13004: invalid_credentials. Nova tentativa em 37.96875s...
Calibração 100: [7/10] Erro 13004: invalid_credentials. Nova tentativa em 56.953125s...
Calibração 100: [8/10] Erro 13004: invalid_credentials. Nova tentativa em 85.4296875s...
Calibração 100: [9/10] Erro 13004: invalid_credentials. Nova tentativa em 128.14453125s...
Calibração 100: [10/10] Erro 13004: invalid_credentials. Nova tentativa em 192.216796875s...
Calibração 100: Não foi possível conectar à Deribit TESTNET após várias tentativas.


Exception: Não foi possível conectar à Deribit TESTNET após várias tentativas.

In [ ]:
import requests

url = "https://test.deribit.com/api/v2/public/auth"
params = {
    "grant_type": "client_credentials",
    "client_id": "w83RPW8w",
    "client_secret": "EfCgKLcX0VsnMpvdGbunv8Pz0CbTAzS7K3H45b6q_YA"
}

response = requests.get(url, params=params)
print("Status:", response.status_code)
print("Resposta:", response.json())

In [ ]:
env_path = "/content/drive/MyDrive/Segredos/.env"

conteudo = """DERIBIT_CLIENT_ID=w83RPW8w
DERIBIT_CLIENT_SECRET=EfCgKLcX0VsnMpvdGbunv8Pz0CbTAzS7K3H45b6q_YA
TELEGRAM_BOT_TOKEN=7589434345:AAF-k9jl7JLAy5I2rgaN1JTJDGd7Dnx7D9c
TELEGRAM_CHAT_ID=5934484407
"""

with open(env_path, "w") as f:
    f.write(conteudo.strip())

print("Arquivo .env atualizado com as novas chaves!")

Arquivo .env atualizado com as novas chaves!


In [ ]:
from dotenv import load_dotenv
load_dotenv("/content/drive/MyDrive/Segredos/.env")

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")
telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

print("Variáveis carregadas!")

Variáveis carregadas!


In [ ]:
token = conectar_deribit_testnet(client_id, client_secret)

Calibração 100: [1/10] Erro 13004: invalid_credentials. Nova tentativa em 5s...
Calibração 100: [2/10] Erro 13004: invalid_credentials. Nova tentativa em 7.5s...
Calibração 100: [3/10] Erro 13004: invalid_credentials. Nova tentativa em 11.25s...
Calibração 100: [4/10] Erro 13004: invalid_credentials. Nova tentativa em 16.875s...
Calibração 100: [5/10] Erro 13004: invalid_credentials. Nova tentativa em 25.3125s...
Calibração 100: [6/10] Erro 13004: invalid_credentials. Nova tentativa em 37.96875s...


KeyboardInterrupt: 

In [ ]:
import requests

url = "https://test.deribit.com/api/v2/public/auth"
params = {
    "grant_type": "client_credentials",
    "client_id": "w83RPW8w",
    "client_secret": "EfCgKLcX0VsnMpvdGbunv8Pz0CbTAzS7K3H45b6q_YA"
}

response = requests.get(url, params=params)
print("Status:", response.status_code)
print("Resposta:", response.json())

Status: 200
Resposta: {'jsonrpc': '2.0', 'result': {'enabled_features': [], 'access_token': '1744693039626.1WuHeDQY.PvLr3YvtRPOfdc_oF8MX2dRhhF4rgQ4V2LmJ787ikl-MovsDJmaWM6hdT_QJDgUsPmC0nN5z7Ogn2lMi828HlH5ZqASe0potgW_0co9xmb0FOgxdesVKT6q6mLL8n5i4nPOp5ftnRv4ebCge-vOD4HC8bRuzSm2Prj4dSKrQMgmn21BTGWv8Hx_u-jm49Qtvm5NpnrHiNXIXeO0A5mWVeoZoY-S1pbrWfoyG9LNlD6mW8k5JA_4HPihXuQ9ycqEgqHI', 'expires_in': 900, 'refresh_token': '1745296939626.1RzzoB3x.aXRMgsLTjetocjfHd8Ps463C123sk0YHa52ln0LwfbOtaZht9i-XdfjxgOLevrPDAwwCma-_0J7jG7dO7t2cDz9tmfPLJoV0wDUsfDoTGvOBo6uw0xIoufqZxL78aQMMagRJUmf9Zte8OTHfLacQo6If16xgpPTtSD6FoNsP4ARv-cA-kBwfJTEmGfHeF7QlR0S44G4XcyVAcjuwYXrBP5KdrO-RKfbdN9FkLncpRymOwDI9fHiYqhbp1eY9wCNnUQ', 'scope': 'session:rest-7UfeV1DJIYg= trade:read_write account:read wallet:read mainaccount', 'sid': '67378.w83RPW8w.rest-7UfeV1DJIYg=', 'token_type': 'bearer'}, 'usIn': 1744692139626411, 'usOut': 1744692139626717, 'usDiff': 306, 'testnet': True}


In [ ]:
Conexão bem-sucedida com a Deribit TESTNET!

SyntaxError: invalid syntax (<ipython-input-28-0e4b41c197be>, line 1)

In [ ]:
from dotenv import load_dotenv
load_dotenv("/content/drive/MyDrive/Segredos/.env")

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")
telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

In [ ]:
def conectar_deribit_testnet_fix(client_id, client_secret, max_tentativas=5, tempo_espera_inicial=5):
    import requests
    import time

    base_url = "https://test.deribit.com/api/v2/public/auth"
    params = {
        "grant_type": "client_credentials",
        "client_id": "w83RPW8w",
        "client_secret": "EfCgKLcX0VsnMpvdGbunv8Pz0CbTAzS7K3H45b6q_YA"
    }

    tentativas = 0
    tempo_espera = tempo_espera_inicial

    while tentativas < max_tentativas:
        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "error" in data:
                msg = f"[{tentativas+1}/{max_tentativas}] Erro {data['error']['code']}: {data['error']['message']}. Nova tentativa em {tempo_espera}s..."
                print("Calibração 100:", msg)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5
            elif "result" in data:
                print("Conexão bem-sucedida com a Deribit TESTNET!")
                return data["result"]["access_token"]
            else:
                print("Erro inesperado:", data)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5
        except Exception as e:
            print("Erro de exceção:", e)
            time.sleep(tempo_espera)
            tentativas += 1
            tempo_espera *= 1.5

    print("Não foi possível conectar após várias tentativas.")

# Executar forçando as credenciais diretamente:
token = conectar_deribit_testnet_fix("w83RPW8w", "EfCgKLcX0VsnMpvdGbunv8Pz0CbTAzS7K3H45b6q_YA")

Conexão bem-sucedida com a Deribit TESTNET!


In [ ]:
import datetime
import requests

# Contador global de ordens
ordens_executadas = 0

# Exemplo de dados simulados (substitua pelas variáveis reais do sistema)
dados_status = {
    "estrategia": "X-Fibra-V2",
    "pnl": 14.25,
    "eficiencia": 0.91,
    "acerto": 0.83,
    "tempo_operacao": 227,  # em minutos
    "tempo_medio_execucao": 7,
    "alocacao": "65%",
    "risco": "Médio",
    "ajuste": "automático"
}

def formatar_tempo(minutos):
    horas = minutos // 60
    minutos = minutos % 60
    return f"{horas}h {minutos}min" if horas > 0 else f"{minutos}min"

def montar_resumo_detalhado():
    agora = datetime.datetime.now().strftime("%H:%M")
    return (
        "[Calibração 100 - Resumo]\n"
        f"Estratégia ativa: {dados_status['estrategia']}\n"
        f"Ordens executadas: {ordens_executadas}\n"
        f"Taxa de acerto: {round(dados_status['acerto'] * 100)}%\n"
        f"PnL acumulado: {dados_status['pnl']:+.2f}\n"
        f"Eficiência média: {round(dados_status['eficiencia'] * 100)}%\n"
        f"Tempo total de operação: {formatar_tempo(dados_status['tempo_operacao'])}\n"
        f"Tempo médio entre execuções: {dados_status['tempo_medio_execucao']}min\n"
        f"Risco atual: {dados_status['risco']}\n"
        f"Alocação atual: {dados_status['alocacao']}\n"
        f"Último ajuste: {dados_status['ajuste']}\n"
        f"Horário do resumo: {agora}"
    )

def enviar_resumo_telegram():
    mensagem = montar_resumo_detalhado()
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    payload = {
        "chat_id": telegram_chat_id,
        "text": mensagem
    }
    try:
        requests.post(url, data=payload)
        print("Resumo enviado com sucesso para o Telegram!")
    except Exception as e:
        print("Erro ao enviar resumo:", e)

In [ ]:
def registrar_ordem():
    global ordens_executadas
    ordens_executadas += 1
    if ordens_executadas % 10 == 0:
        enviar_resumo_telegram()

In [ ]:
import threading

def agendar_resumo_diario():
    def checar_horario():
        while True:
            agora = datetime.datetime.now()
            if agora.strftime("%H:%M") == "18:00":
                enviar_resumo_telegram()
                time.sleep(60)  # espera 1 minuto para não enviar múltiplos
            time.sleep(10)

    thread = threading.Thread(target=checar_horario)
    thread.daemon = True
    thread.start()

# Ativar monitoramento diário
agendar_resumo_diario()

In [ ]:
import datetime
import requests
import csv
import os

# Variáveis reais (atualize dinamicamente no seu sistema)
estrategia_ativa = "X-Fibra-V2"
pnl_total = 14.25
eficiencia_media = 0.91
taxa_acerto = 0.83
tempo_total_operacao_em_minutos = 227
tempo_medio_execucao = 7
alocacao_atual = "65%"
risco_atual = "Médio"
ajuste_ativo = "automático"
ordens_executadas = 30  # este número deve ser atualizado pelo seu sistema

# Obter dados reais do sistema
def obter_dados_status_reais():
    return {
        "estrategia": estrategia_ativa,
        "pnl": pnl_total,
        "eficiencia": eficiencia_media,
        "acerto": taxa_acerto,
        "tempo_operacao": tempo_total_operacao_em_minutos,
        "tempo_medio_execucao": tempo_medio_execucao,
        "alocacao": alocacao_atual,
        "risco": risco_atual,
        "ajuste": ajuste_ativo
    }

# Formatar tempo
def formatar_tempo(minutos):
    horas = minutos // 60
    minutos = minutos % 60
    return f"{horas}h {minutos}min" if horas > 0 else f"{minutos}min"

# Montar resumo detalhado
def montar_resumo_detalhado():
    status = obter_dados_status_reais()
    agora = datetime.datetime.now().strftime("%H:%M")
    return (
        "[Calibração 100 - Resumo]\n"
        f"Estratégia ativa: {status['estrategia']}\n"
        f"Ordens executadas: {ordens_executadas}\n"
        f"Taxa de acerto: {round(status['acerto'] * 100)}%\n"
        f"PnL acumulado: {status['pnl']:+.2f}\n"
        f"Eficiência média: {round(status['eficiencia'] * 100)}%\n"
        f"Tempo total de operação: {formatar_tempo(status['tempo_operacao'])}\n"
        f"Tempo médio entre execuções: {status['tempo_medio_execucao']}min\n"
        f"Risco atual: {status['risco']}\n"
        f"Alocação atual: {status['alocacao']}\n"
        f"Último ajuste: {status['ajuste']}\n"
        f"Horário do resumo: {agora}"
    )

# Enviar para o Telegram
def enviar_resumo_telegram():
    mensagem = montar_resumo_detalhado()
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    payload = {
        "chat_id": telegram_chat_id,
        "text": mensagem
    }
    try:
        requests.post(url, data=payload)
        print("Resumo enviado para o Telegram!")
        salvar_resumo_em_csv()
    except Exception as e:
        print("Erro ao enviar resumo:", e)

# Salvar no histórico CSV
caminho_csv = "/content/drive/MyDrive/Calibracao100_resumos.csv"

def salvar_resumo_em_csv():
    status = obter_dados_status_reais()
    agora = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    cabecalho = [
        "Data/Hora", "Estratégia", "Ordens", "Acerto (%)", "PnL", "Eficiência (%)",
        "Tempo Total (min)", "Médio Execução", "Risco", "Alocação", "Ajuste"
    ]
    dados = [
        agora, status["estrategia"], ordens_executadas, round(status["acerto"]*100),
        round(status["pnl"], 2), round(status["eficiencia"]*100),
        status["tempo_operacao"], status["tempo_medio_execucao"],
        status["risco"], status["alocacao"], status["ajuste"]
    ]

    file_exists = os.path.isfile(caminho_csv)
    with open(caminho_csv, mode="a", newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(cabecalho)
        writer.writerow(dados)

    print("Resumo salvo no histórico CSV.")

In [ ]:
def conectar_deribit(client_id, client_secret, max_tentativas=5, tempo_espera_inicial=5):
    base_url = "https://test.deribit.com/api/v2/public/auth"  # Corrigido para testnet
    params = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    tentativas = 0
    tempo_espera = tempo_espera_inicial

    while tentativas < max_tentativas:
        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "error" in data:
                msg = f"[{tentativas+1}/{max_tentativas}] Erro {data['error']['code']}: {data['error']['message']}. Nova tentativa em {tempo_espera}s..."
                print("Calibração 100:", msg)
                enviar_alerta_telegram(msg)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5
            elif "result" in data:
                print("Conexão bem-sucedida com a Deribit TESTNET!")
                enviar_alerta_telegram("Conexão bem-sucedida com a Deribit TESTNET!")
                return data["result"]["access_token"]
            else:
                msg = f"[{tentativas+1}] Erro inesperado: {data}"
                print("Calibração 100:", msg)
                enviar_alerta_telegram(msg)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5
        except Exception as e:
            msg = f"[{tentativas+1}] Erro de exceção: {e}"
            print("Calibração 100:", msg)
            enviar_alerta_telegram(msg)
            time.sleep(tempo_espera)
            tentativas += 1
            tempo_espera *= 1.5

    erro_final = "Não foi possível conectar à Deribit TESTNET após várias tentativas."
    print("Calibração 100:", erro_final)
    enviar_alerta_telegram(erro_final)
    raise Exception(erro_final)

In [ ]:
def conectar_deribit(client_id, client_secret, max_tentativas=5, tempo_espera_inicial=5):
    base_url = "https://test.deribit.com/api/v2/public/auth"  # <- TESTNET!
    params = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    tentativas = 0
    tempo_espera = tempo_espera_inicial

    while tentativas < max_tentativas:
        try:
            response = requests.get(base_url, params=params)
            data = response.json()

            if "error" in data:
                msg = f"[{tentativas+1}/{max_tentativas}] Erro {data['error']['code']}: {data['error']['message']}. Nova tentativa em {tempo_espera}s..."
                print("Calibração 100:", msg)
                enviar_alerta_telegram(msg)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5
            elif "result" in data:
                print("Conexão bem-sucedida com a Deribit TESTNET!")
                enviar_alerta_telegram("Conexão bem-sucedida com a Deribit TESTNET!")
                return data["result"]["access_token"]
            else:
                msg = f"[{tentativas+1}] Erro inesperado: {data}"
                print("Calibração 100:", msg)
                enviar_alerta_telegram(msg)
                time.sleep(tempo_espera)
                tentativas += 1
                tempo_espera *= 1.5
        except Exception as e:
            msg = f"[{tentativas+1}] Erro de exceção: {e}"
            print("Calibração 100:", msg)
            enviar_alerta_telegram(msg)
            time.sleep(tempo_espera)
            tentativas += 1
            tempo_espera *= 1.5

    erro_final = "Não foi possível conectar à Deribit TESTNET após várias tentativas."
    print("Calibração 100:", erro_final)
    enviar_alerta_telegram(erro_final)
    raise Exception(erro_final)

In [ ]:

import os

def verificar_urls_mainnet():
    caminho_notebook = "/content"  # Pasta raiz onde Colab armazena os arquivos
    mainnet_urls = []

    for root, dirs, files in os.walk(caminho_notebook):
        for file in files:
            if file.endswith(".ipynb"):
                caminho_completo = os.path.join(root, file)
                with open(caminho_completo, "r", encoding="utf-8") as f:
                    conteudo = f.read()
                    if "www.deribit.com" in conteudo:
                        mainnet_urls.append(caminho_completo)

    if mainnet_urls:
        print("ATENÇÃO! Foram encontradas URLs da MAINNET nestes notebooks:")
        for caminho in mainnet_urls:
            print(" -", caminho)
        print("\nSubstitua todas por: test.deribit.com")
    else:
        print("✅ Nenhuma URL da mainnet foi encontrada. Está 100% na TESTNET!")

verificar_urls_mainnet()

FileNotFoundError: [Errno 2] No such file or directory

In [ ]:
import os

def verificar_urls_mainnet():
    caminho_notebook = "/content"
    mainnet_urls = []

    for root, dirs, files in os.walk(caminho_notebook):
        for file in files:
            if file.endswith(".ipynb"):
                caminho_completo = os.path.join(root, file)
                try:
                    with open(caminho_completo, "r", encoding="utf-8") as f:
                        conteudo = f.read()
                        if "www.deribit.com" in conteudo:
                            mainnet_urls.append(caminho_completo)
                except Exception as e:
                    print(f"Ignorando {file} (erro ao abrir): {e}")

    if mainnet_urls:
        print("ATENÇÃO! Foram encontradas URLs da MAINNET nestes notebooks:")
        for caminho in mainnet_urls:
            print(" -", caminho)
        print("\nSubstitua todas por: test.deribit.com")
    else:
        print("✅ Nenhuma URL da mainnet foi encontrada. Está 100% na TESTNET!")

verificar_urls_mainnet()

Ignorando Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Untitled4.ipynb (erro ao abrir): [Errno 2] No such file or directory
ATENÇÃO! Foram encontradas URLs da MAINNET nestes notebooks:
 - /content/drive/MyDrive/Colab Notebooks/Cópia de Cópia de Cópia de Cópia de Cópia de Untitled4.ipynb
 - /content/drive/MyDrive/Colab Notebooks/Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Untitled4 (1).ipynb
 - /content/drive/MyDrive/Colab Notebooks/Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Untitled4 (1).ipynb
 - /content/drive/MyDrive/Colab Notebooks/Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Untitled4.ipynb
 - /content/drive/MyDrive/Colab Notebooks/Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Untitled4.ipynb
 - /content/drive/MyDrive/Colab Notebooks/Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Untitled4 (1).ipynb


In [ ]:
import os
import re

def verificar_credenciais_hardcoded():
    termos_proibidos = [
        r'client_id\s*=\s*["\'].*["\']',
        r'client_secret\s*=\s*["\'].*["\']'
    ]
    caminho_root = "/content"
    encontrados = []

    for root, dirs, files in os.walk(caminho_root):
        for file in files:
            if file.endswith(".ipynb"):
                caminho_completo = os.path.join(root, file)
                try:
                    with open(caminho_completo, "r", encoding="utf-8") as f:
                        conteudo = f.read()
                        for termo in termos_proibidos:
                            if re.search(termo, conteudo):
                                encontrados.append(caminho_completo)
                                break
                except Exception as e:
                    print(f"Ignorando {file} (erro ao abrir): {e}")

    if encontrados:
        print("⚠️ Credenciais hardcoded encontradas nestes notebooks:")
        for caminho in encontrados:
            print(" -", caminho)
        print("\nRecomendo substituir por os.getenv('DERIBIT_CLIENT_ID') e usar o .env.")
    else:
        print("✅ Nenhum client_id/client_secret fixo encontrado. Está seguro!")

verificar_credenciais_hardcoded()

Ignorando Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Cópia de Untitled4.ipynb (erro ao abrir): [Errno 2] No such file or directory
✅ Nenhum client_id/client_secret fixo encontrado. Está seguro!


In [ ]:
token = conectar_deribit(client_id, client_secret)

Calibração 100: [1/5] Erro 13004: invalid_credentials. Nova tentativa em 5s...
Calibração 100: [2/5] Erro 13004: invalid_credentials. Nova tentativa em 7.5s...
Calibração 100: [3/5] Erro 13004: invalid_credentials. Nova tentativa em 11.25s...
Calibração 100: [4/5] Erro 13004: invalid_credentials. Nova tentativa em 16.875s...
Calibração 100: [5/5] Erro 13004: invalid_credentials. Nova tentativa em 25.3125s...
Calibração 100: Não foi possível conectar à Deribit TESTNET após várias tentativas.


Exception: Não foi possível conectar à Deribit TESTNET após várias tentativas.

In [ ]:
token = conectar_deribit_testnet(client_id, client_secret)

Calibração 100: [1/10] Erro 13004: invalid_credentials. Nova tentativa em 5s...
Calibração 100: [2/10] Erro 13004: invalid_credentials. Nova tentativa em 7.5s...
Calibração 100: [3/10] Erro 13004: invalid_credentials. Nova tentativa em 11.25s...
Calibração 100: [4/10] Erro 13004: invalid_credentials. Nova tentativa em 16.875s...
Calibração 100: [5/10] Erro 13004: invalid_credentials. Nova tentativa em 25.3125s...
Calibração 100: [6/10] Erro 13004: invalid_credentials. Nova tentativa em 37.96875s...


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Caminho onde o .env será salvo (na pasta "Segredos" do seu Drive)
env_path = "/content/drive/MyDrive/Segredos/.env"

# Substitua os valores abaixo pelas suas chaves reais
conteudo = """DERIBIT_CLIENT_ID=QHy4D9s-
DERIBIT_CLIENT_SECRET=y47wloxUzjTKRoCkr-9H5oUW8l0sgOdG3yzvE165EIU
TELEGRAM_BOT_TOKEN=7589434345:AAF-k9jl7JLAy5I2rgaN1JTJDGd7Dnx7D9c
TELEGRAM_CHAT_ID=5934484407
"""

# Criar e salvar o arquivo .env
with open(env_path, "w") as f:
    f.write(conteudo.strip())

print("Arquivo .env criado com sucesso em:", env_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Segredos/.env'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

env_path = "/content/drive/MyDrive/Segredos/.env"
conteudo = """DERIBIT_CLIENT_ID=QHy4D9s-
DERIBIT_CLIENT_SECRET=y47wloxUzjTKRoCkr-9H5oUW8l0sgOdG3yzvE165EIU
TELEGRAM_BOT_TOKEN=7589434345:AAF-k9jl7JLAy5I2rgaN1JTJDGd7Dnx7D9c
TELEGRAM_CHAT_ID=5934484407
"""

with open(env_path, "w") as f:
    f.write(conteudo.strip())

print("Arquivo .env criado com sucesso em:", env_path)


Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Segredos/.env'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

env_path = "/content/drive/MyDrive/Segredos/.env"
conteudo = """DERIBIT_CLIENT_ID=sua_client_id
DERIBIT_CLIENT_SECRET=seu_client_secret
TELEGRAM_BOT_TOKEN=seu_bot_token
TELEGRAM_CHAT_ID=seu_chat_id
"""

with open(env_path, "w") as f:
    f.write(conteudo.strip())

print("Arquivo .env criado com sucesso em:", env_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Segredos/.env'

In [ ]:
!pip install -q python-dotenv

import os
from dotenv import load_dotenv

env_path = '/content/drive/MyDrive/Segredos/.env'
load_dotenv(env_path)

client_id = os.getenv("DERIBIT_CLIENT_ID")
client_secret = os.getenv("DERIBIT_CLIENT_SECRET")
telegram_token = os.getenv("TELEGRAM_BOT_TOKEN")
telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")

if client_id and client_secret:
    print("Chaves carregadas com sucesso.")
else:
    print("Erro ao carregar as chaves.")


In [ ]:
import requests
import time

def enviar_alerta_telegram(msg):
    if not telegram_token or not telegram_chat_id:
        print("Telegram não configurado.")
        return
    url = f"https://api.telegram.org/bot{telegram_token}/sendMessage"
    payload = {
        "chat_id": telegram_chat_id,
        "text": msg
    }
    try:
        requests.post(url, data=payload)
    except Exception as e:
        print("Erro ao enviar alerta:", e)

def conectar_deribit(client_id, client_secret, max_tentativas=5):
    url = "https://www.deribit.com/api/v2/public/auth"
    payload = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }

    for tentativa in range(max_tentativas):
        try:
            response = requests.post(url, data=payload)
            data = response.json()

            if "error" in data and data["error"]["code"] == 13004:
                msg = f"[{tentativa+1}] Erro: invalid_credentials. Tentando em 5s..."
                print(msg)
                enviar_alerta_telegram(msg)
                time.sleep(5)
            elif "result" in data:
                print("Conexão bem-sucedida com a Deribit!")
                return data["result"]["access_token"]
            else:
                print("Erro inesperado:", data)
                break
        except Exception as e:
            print(f"[{tentativa+1}] Erro ao conectar: {e}")
            enviar_alerta_telegram(f"Erro ao conectar: {e}")
            time.sleep(5)

    msg = "Não foi possível conectar à Deribit após várias tentativas."
    enviar_alerta_telegram(msg)
    raise Exception(msg)

token = conectar_deribit(client_id, client_secret)
